In [1]:
from __future__ import print_function

import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

In [7]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('calendar', 'v3', credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId='primary', timeMin=now,
                                            singleEvents=True,
                                            orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')

    # Prints the start and name of the next 10 events
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(start, event['summary'])

except HttpError as error:
    print('An error occurred: %s' % error)


Getting the upcoming 10 events
2022-06-14 Ritiro Confirmation Letter Erasmum office
2022-06-15 Biosensors Exam 1st call
2022-06-15T11:30:00+02:00 Deadline Review NearLab paper
2022-06-17 EXAM: Sensors from h13.30
2022-06-30 Neuroengineering Exam 1st call
2022-07-01 Computational Biomechanics Exam 1st call
2022-07-05 Eliminazione Instagram
2022-07-18 Computational Biomechanics Exam 2nd call
2022-07-20 Laurea Ventu
2022-07-21 Neuroengineering Exam 2nd call
2022-07-26 Biosensors Exam 2nd call


In [6]:
from telegram.ext import (Updater, 
    CommandHandler,
    MessageHandler,
    CallbackQueryHandler,
    ConversationHandler,
    CallbackContext,    
    Filters
)
import datetime

In [7]:
with open("telegramapi.txt") as f:
        tokens = f.readlines()
bot_token = tokens[0].strip("\n")
updater = Updater(token=bot_token, use_context=True)
        


In [8]:
        
print(">> "+datetime.datetime.now().strftime("[%d/%m/%Y-%H:%M:%S]-BOT: ")+
        " Loading Telegram bot: ",end="")
dispatcher = updater.dispatcher
print("SUCCESS") if updater!=0 else print("FAIL")


>> [13/06/2022-16:14:27]-BOT:  Loading Telegram bot: SUCCESS


In [9]:
def send_message(context):
    user_id = 410773404


    context.bot.send_message(
        chat_id=user_id,
        text="Enculet!"
    )

def printid(update, context):
    userid = update.message.from_user['id']
    print(userid)
    context.bot.send_message(
        chat_id=update.effective_chat.id, 
        text=userid
    )


dispatcher.add_handler(CommandHandler('send_message', send_message))
dispatcher.add_handler(CommandHandler('start', printid))

jobqueue = updater.job_queue
jobqueue.run_repeating(
    callback=send_message,
    interval=1,
)


In [10]:

updater.start_polling()
updater.idle()